<a href="https://colab.research.google.com/github/MariaOmaechevarria/OPENCL/blob/main/OBTENER_INFORMACION_DEL_DEVICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''

!sudo apt update
!sudo apt purge *nvidia* -y
!sudo apt install nvidia-driver-530 -y


!pip install pyopencl
!apt-get install -y pocl-opencl-icd ocl-icd-libopencl1
'''

'\n\n!sudo apt update\n!sudo apt purge *nvidia* -y\n!sudo apt install nvidia-driver-530 -y\n\n\n!pip install pyopencl\n!apt-get install -y pocl-opencl-icd ocl-icd-libopencl1\n'

In [2]:
import pyopencl as cl

# Configurar contexto y seleccionar el dispositivo
platform = cl.get_platforms()[0]  # Selecciona la primera plataforma
device = platform.get_devices()[0]  # Selecciona el primer dispositivo

# Obtener el tipo de dispositivo
device_type = device.get_info(cl.device_info.TYPE)

if device_type == cl.device_type.CPU:
    print("CL_DEVICE_TYPE_CPU")
elif device_type == cl.device_type.GPU:
    print("CL_DEVICE_TYPE_GPU")
elif device_type == cl.device_type.ACCELERATOR:
    print("CL_DEVICE_TYPE_ACCELERATOR")
else:
    print("Other")

# Obtener el Vendor ID del dispositivo
vendor_id = device.get_info(cl.device_info.VENDOR_ID)
print(f"Vendor ID: 0x{vendor_id:04x}")
device_name = device.get_info(cl.device_info.NAME)
print(f"Nombre del dispositivo: {device_name}")


# Obtener el número máximo de Compute Units
max_compute_units = device.get_info(cl.device_info.MAX_COMPUTE_UNITS)
print(f"Maximum Compute Units: {max_compute_units}")

# Obtener las dimensiones máximas de trabajo
max_work_item_dimensions = device.get_info(cl.device_info.MAX_WORK_ITEM_DIMENSIONS)
print(f"Maximum Work Item Dimensions: {max_work_item_dimensions}")

# Obtener los tamaños máximos de trabajo
max_work_item_sizes = device.get_info(cl.device_info.MAX_WORK_ITEM_SIZES)
print(f"Maximum Work Item Sizes: {max_work_item_sizes[0]} x {max_work_item_sizes[1]} x {max_work_item_sizes[2]}")

# Obtener el tamaño máximo del grupo de trabajo
max_work_group_size = device.get_info(cl.device_info.MAX_WORK_GROUP_SIZE)
print(f"Maximum Work Group Size: {max_work_group_size}")

# Obtener la frecuencia máxima del reloj
max_clock_frequency = device.get_info(cl.device_info.MAX_CLOCK_FREQUENCY)
print(f"Maximum Clock Frequency: {max_clock_frequency} MHz")


CL_DEVICE_TYPE_GPU
Vendor ID: 0x8086
Nombre del dispositivo: Intel(R) Iris(R) Xe Graphics
Maximum Compute Units: 96
Maximum Work Item Dimensions: 3
Maximum Work Item Sizes: 512 x 512 x 512
Maximum Work Group Size: 512
Maximum Clock Frequency: 1250 MHz


In [3]:
!clinfo


"clinfo" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [4]:
import pyopencl as cl

# Obtiene la primera plataforma y el primer dispositivo disponibles
platforms = cl.get_platforms()
if not platforms:
    raise RuntimeError("No se encontraron plataformas OpenCL disponibles.")
platform = platforms[0]

devices = platform.get_devices()
if not devices:
    raise RuntimeError("No se encontraron dispositivos OpenCL en la plataforma seleccionada.")
device = devices[0]

# Crea un contexto y una cola de comando
ctx = cl.Context([device])
queue = cl.CommandQueue(ctx)

# Un kernel sencillo de ejemplo (suma de dos vectores)
program_source = """__kernel void test_kernel(int dim, __global int* A, __global int* B, __global int* C) {
    //Obtener IDs del work item
    int fila = get_global_id(0);
    int columna = get_global_id(1);
    
    //Inicializar resultado
    int resultado = 0;
    
    //Recorrer la fila y columna correspondiente
    for (int i = 0; i < dim; i++) {
        resultado += A[fila * dim + i] * B[i * dim + columna];
    }
    
    //Establecer resultado en C
    C[fila * dim + columna] = resultado;
}
"""

# Compila el programa
program = cl.Program(ctx, program_source).build()

# Crea el kernel
kernel = cl.Kernel(program, "test_kernel")

# Obtiene el tamaño máximo de work group soportado para este kernel en el dispositivo
max_work_group_size = kernel.get_work_group_info(cl.kernel_work_group_info.WORK_GROUP_SIZE, device)

print("Tamaño máximo de work group soportado para este kernel:", max_work_group_size)


Tamaño máximo de work group soportado para este kernel: 512
